## Traducción automática basado en reglas en-es

#### Cargando modelos en-es

In [2]:
!python -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 66.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [3]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 72.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [5]:
import spacy
nlp_es = spacy.load('es_core_news_sm')
nlp_en = spacy.load('en_core_web_sm')

#### Analizador morfológico

In [6]:
def analizador_morfologico(texto, idioma="es"):
    if idioma == "es":
        doc = nlp_es(texto)
    else:
        doc = nlp_en(texto)

    morfologia = []
    for token in doc:
        morfologia.append({
            "texto": token.text,
            "lema": token.lemma_,
            "pos": token.pos_,
            "etiqueta": token.tag_,
            "morf": token.morph
        })
    return morfologia

#### Análisis sintáctico

In [7]:
def analizador_sintactico(texto, idioma="es"):
    if idioma == "es":
        doc = nlp_es(texto)
    else:
        doc = nlp_en(texto)

    sintaxis = []
    for token in doc:
        sintaxis.append({
            "texto": token.text,
            "lema": token.lemma_,
            "pos": token.pos_,
            "etiqueta": token.tag_,
            "dependencia": token.dep_,
            "cabeza": token.head.text
        })
    return sintaxis

#### Definición de diccinarios

In [8]:
translation_dict = {
    "gato": "cat",
    "perro": "dog",
    "mascota": "pet",
    "parque": "park",
    "los": "the",
    "las": "the",
    "el": "the",
    "la": "the",
    "un": "a",
    "una": "a",
    "unos": "some",
    "unas": "some"
}


#### Traduccón en-es y es-en

In [9]:
def traducir(texto, source_lang="es", target_lang="en"):
    morfologia = analizador_morfologico(texto, idioma=source_lang)
    sintaxis = analizador_sintactico(texto, idioma=source_lang)
    traduccion = []
    for word_info in morfologia:
        lemma = word_info["lema"].lower()

        if source_lang == "es" and lemma in translation_dict:
            translated_word = translation_dict[lemma]
            if word_info["morf"].get("Number") == ["Plur"]:
                if not translated_word.endswith("s"):
                    translated_word += "s"
            traduccion.append(translated_word)

        elif source_lang == "en" and lemma in translation_dict.values():
            translated_word = [key for key, value in translation_dict.items() if value == lemma][0]
            if word_info["morf"].get("Number") == ["plur"]:
                if translated_word.endswith("o"):
                    translated_word = translated_word[:-1] + "os"
                elif translated_word.endswith("a"):
                    translated_word = translated_word[:-1] + "as"
                else:
                    translated_word += "s"
            traduccion.append(translated_word)
        else:
            traduccion.append(word_info["texto"])

    if source_lang == "en":
        for i in range(len(traduccion)):
            if traduccion[i] == "the":
                if i+1 < len(traduccion):
                    next_word = traduccion[i+1]
                    if next_word.endswith("s"):
                        if next_word.endswith("as"):
                            traduccion[i] = "las"
                        else:
                            traduccion[i] = "los"
                    else:
                        if next_word.endswith("a"):
                            traduccion[i] = "la"
                        else:
                            traduccion[i] = "el"

    return " ".join(traduccion)

#### Ejemplo de entrada y salida de la traducción

In [10]:
texto_espanol = "El gato y el perro son mascotas."
traduccion_a_ingles = traducir(texto_espanol, source_lang="es", target_lang="en")
print(f"Español: {texto_espanol}")
print(f"Inglés: {traduccion_a_ingles}")

print()

texto_ingles = "The pets are in the park."
traduccion_a_espanol = traducir(texto_ingles, source_lang="en", target_lang="es")
print(f"Inglés: {texto_ingles}")
print(f"Español: {traduccion_a_espanol}")


Español: El gato y el perro son mascotas.
Inglés: the cat y the dog son pets .

Inglés: The pets are in the park.
Español: los mascota are in los parque .
